In [50]:
import ipyplot
%matplotlib notebook
import matplotlib.pyplot as plt

import rawpy 
import cv2
import numpy as np


import os
import os.path as op


data_dir = op.join("..", "data")
img_name = "DSC01088"
raw_fp = op.join(data_dir, img_name+".ARW")

assert op.exists(raw_fp)

In [81]:
raw = rawpy.imread(raw_fp)

a = raw.postprocess(half_size=True, no_auto_bright=True, no_auto_scale=True, output_color=rawpy.ColorSpace.raw, output_bps=16)[:,:,1].copy()/2**16
b = raw.postprocess(half_size=True, no_auto_bright=True, no_auto_scale=True, output_color=rawpy.ColorSpace.raw, output_bps=16)[:,:,1].copy()/2**16

In [82]:
def superpose(img, bins_n):
    ref = img.copy()
    t_mins = [b*1/bins_n for b in range(bins_n)]
    t_maxs = [b*1/bins_n for b in range(1,bins_n+1)]
    t_maxs[-1] += 1e-6
    for t_min, t_max in zip(t_mins, t_maxs):
        idx = np.where(np.logical_and(ref>=t_min, ref<t_max))
        img[idx] = (img[idx] - t_min) / (t_max - t_min)
    return img

In [83]:
result=superpose(b,10)
result.max()

0.9997253417968751

In [84]:
np.histogram(a, bins=[b*1/5 for b in range(5)], density=True)

(array([4.87633708, 0.10393279, 0.01973013, 0.        ]),
 array([0. , 0.2, 0.4, 0.6, 0.8]))

In [85]:
np.histogram(result, bins=[b*1/5 for b in range(5)], density=True)

(array([3.01201641, 0.80498256, 0.68278036, 0.50022068]),
 array([0. , 0.2, 0.4, 0.6, 0.8]))

In [86]:
cv2.imwrite("a.jpg", a*255)
cv2.imwrite("result.jpg", result*255)

True